In [125]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/INTER_IIT_2024_DATA")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
!ls

 BTC_2019_2023_12h.csv		     BTC_2019_2023_30m.csv  'ETHUSDT_1d (1).csv'
 BTC_2019_2023_15m.csv		     BTC_2019_2023_3d.csv    ETHUSDT_1h.csv
'BTC_2019_2023_1d (1).csv'	     BTC_2019_2023_3m.csv    ETHUSDT_2h.csv
 BTC_2019_2023_1d.csv		     BTC_2019_2023_4h.csv    ETHUSDT_30m.csv
 BTC_2019_2023_1d_with_signals.csv   BTC_2019_2023_6h.csv    ETHUSDT_3m.csv
'BTC_2019_2023_1h (1).csv'	     BTC_2019_2023_8h.csv    ETHUSDT_4h.csv
 BTC_2019_2023_1h.csv		     df_1m.csv		     ETHUSDT_5m.csv
 BTC_2019_2023_1m.csv		     df_download.csv	     ETHUSDT_6h.csv
 BTC_2019_2023_1month.csv	     downloaded_df.csv	     ETHUSDT_8h.csv
 BTC_2019_2023_1w.csv		     ETHUSDT_12h.csv	     result_df.csv
'BTC_2019_2023_2h (2).csv'	     ETHUSDT_15m.csv	     untrade-sdk


In [127]:
import numpy as np
import pandas as pd

In [128]:
df= pd.read_csv('ETHUSDT_4h.csv') #use 4h ethereum data

In [129]:
df['datetime'] = pd.to_datetime(df['datetime'])  # Convert 'date' column to datetime objects if it's not already

# Keep only rows until 2020-01-01 00:00:00
df = df[df['datetime'] >= '2020-01-01 00:00:00']

# Reset the index if needed
df = df.reset_index(drop=True)

In [130]:
df=df[df.volume!=0]

In [131]:
df = df.reset_index(drop=True)

In [132]:
def total_signal(df, current_candle):
    current_pos = df.index.get_loc(current_candle)

    # Buy condition
    c1 = df['high'].iloc[current_pos] > df['high'].iloc[current_pos-1]
    c2 = df['high'].iloc[current_pos-1] > df['low'].iloc[current_pos]
    c3 = df['low'].iloc[current_pos] > df['high'].iloc[current_pos-2]
    c4 = df['high'].iloc[current_pos-2] > df['low'].iloc[current_pos-1]
    c5 = df['low'].iloc[current_pos-1] > df['high'].iloc[current_pos-3]
    c6 = df['high'].iloc[current_pos-3] > df['low'].iloc[current_pos-2]
    c7 = df['low'].iloc[current_pos-2] > df['low'].iloc[current_pos-3]

    if c1 and c2 and c3 and c4 and c5 and c6 and c7:
        return 1

    # Symmetrical conditions for short (sell condition)
    c1 = df['low'].iloc[current_pos] < df['low'].iloc[current_pos-1]
    c2 = df['low'].iloc[current_pos-1] < df['high'].iloc[current_pos]
    c3 = df['high'].iloc[current_pos] < df['low'].iloc[current_pos-2]
    c4 = df['low'].iloc[current_pos-2] < df['high'].iloc[current_pos-1]
    c5 = df['high'].iloc[current_pos-1] < df['low'].iloc[current_pos-3]
    c6 = df['low'].iloc[current_pos-3] < df['high'].iloc[current_pos-2]
    c7 = df['high'].iloc[current_pos-2] < df['high'].iloc[current_pos-3]

    if c1 and c2 and c3 and c4 and c5 and c6 and c7:
        return -1

    return 0


In [133]:

df['signals'] = df.apply(lambda row: total_signal(df, row.name), axis=1)

In [134]:
data=df.copy()

In [135]:
data["TP"] = np.where(
         data["signals"] > 0, data["close"] * 1.1, 0
     )  # target price aboveclose price for long positions
data["SL"] = np.where(
         data["signals"] < 0, data["close"] * 0.90, 0
     )  # stop loss below the close price for short positions


In [136]:
cols = list(data.columns)
cols.remove('SL')
cols.remove('signals')
cols.remove('TP')
data = data[cols + ['TP','SL', 'signals']]
data

,Unnamed: 0,datetime,open,high,low,close,volume,TP,SL,signals
0,186,2020-01-01 00:00:00,129.12,130.97,128.62,130.09,88391.064,0.0,0.0,0
1,187,2020-01-01 04:00:00,130.10,130.67,130.00,130.17,46813.723,0.0,0.0,0
2,188,2020-01-01 08:00:00,130.15,131.88,129.75,130.64,70366.957,0.0,0.0,0
3,189,2020-01-01 12:00:00,130.63,132.49,130.63,131.97,75684.307,0.0,0.0,0
4,190,2020-01-01 16:00:00,132.00,132.96,131.45,131.79,78724.484,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
8762,8948,2023-12-31 08:00:00,2296.95,2318.00,2291.43,2310.66,324174.135,0.0,0.0,0
8763,8949,2023-12-31 12:00:00,2310.66,2324.78,2291.86,2300.60,358689.484,0.0,0.0,0
8764,8950,2023-12-31 16:00:00,2300.59,2311.74,2292.56,2296.86,226419.616,0.0,0.0,0
8765,8951,2023-12-31 20:00:00,2296.85,2296.91,2258.00,2283.85,360287.848,0.0,0.0,0


In [137]:

data.dtypes

,0
Unnamed: 0,int64
datetime,datetime64[ns]
open,float64
high,float64
low,float64
close,float64
volume,float64
TP,float64
SL,float64
signals,int64


In [138]:
import pandas as pd

def filter_trade_signals(df):
    cumulative_position = 0

    df['new_signals'] = df['signals']

    for i in range(len(df)):
        current_signal = df.loc[i, 'new_signals']

        if current_signal != 0:
            if cumulative_position == 0:
                cumulative_position = current_signal
            elif (cumulative_position > 0 and current_signal == -1) or (cumulative_position < 0 and current_signal == 1):
                cumulative_position = 0  # Close the position

            else:
                df.loc[i, 'new_signals'] = 0
        else:

            continue

    return df

df = filter_trade_signals(df)

In [139]:
if 'new_signals' in df.columns:
  df['signals'] = df['new_signals']
  df = df.drop('new_signals', axis=1)

In [140]:
from google.colab import files

data.to_csv('result_df.csv', index=False)
files.download('result_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
import os
import sys
!git clone https://github.com/ztuntrade/untrade-sdk.git
os.chdir('untrade-sdk')
!pip3 install .
untrade1= os.path.abspath('untrade-sdk')
sys.path.append(untrade1)
from untrade.client import Client
client = Client()

fatal: destination path 'untrade-sdk' already exists and is not an empty directory.
Processing /content/drive/MyDrive/INTER_IIT_2024_DATA/untrade-sdk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5079 sha256=16566ba2cb84de9ead5673f93ab1907645436216ee30105c4887ae6208b033d5
  Stored in directory: /root/.cache/pip/wheels/2c/3c/fa/157295858821e9c5f1508f2f3473c1d1209bf7475ee9425b03
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      Successfully uninstalled untrade-0.1.0


In [142]:
import pandas as pd
from pprint import pprint
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np

In [145]:
# Function to plot trade signals on a candlestick chart
def plot_signals(csv_file):

    # Load data from the CSV file
    data = pd.read_csv(csv_file, index_col='datetime', parse_dates=True)

    # Map trade types for better visualization and debugging
    data['trade_type'] = data['signals'].map({
        1: 'Long',
        -1: 'Short',
        0: 'Hold',
        2: 'Long Reversal',
        -2: 'Short Reversal'
    })

    # Create a figure for the chart
    fig = go.Figure()

    # Plot the price data as a candlestick chart
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        name='Candlestick',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Plot Long (1) signals as green upward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 1],
        y=data['close'][data['signals'] == 1],
        mode='markers',
        marker=dict(symbol='triangle-up', color='lime', size=10, line=dict(color='black', width=1)),
        name='Long (1)'
    ))

    # Plot Short (-1) signals as red downward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -1],
        y=data['close'][data['signals'] == -1],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10, line=dict(color='black', width=1)),
        name='Short (-1)'
    ))

    # Plot Long Reversal (2) signals as cyan stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 2],
        y=data['close'][data['signals'] == 2],
        mode='markers',
        marker=dict(symbol='star', color='cyan', size=10, line=dict(color='black', width=1)),
        name='Long Reversal (2)'
    ))

    # Plot Short Reversal (-2) signals as magenta stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -2],
        y=data['close'][data['signals'] == -2],
        mode='markers',
        marker=dict(symbol='star', color='magenta', size=10, line=dict(color='black', width=1)),
        name='Short Reversal (-2)'
    ))

    # Update layout to add titles and improve visualization
    fig.update_layout(
        title=f'ETHEREUM Candlestick Chart with Trade Signals',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False,
        legend=dict(y=1, traceorder='normal'),
        margin=dict(l=0, r=0, t=50, b=50),
    )

    # Display the chart
    fig.show()

# Function to calculate PnL and related data from a CSV file
def calculate_pnl_data(csv_file_path, initial_capital=1000):
    # Load data from the CSV file
    data = pd.read_csv(csv_file_path)

    # Initialize PnL-related columns
    data['entry'] = 0.0
    data['exit'] = 0.0
    data['quantity'] = 0.0
    data['pnl'] = 0.0
    data['Transaction_Cost'] = 0.0
    data['Net_Pnl'] = 0.0
    data['drawdown'] = 0.0
    trade_type = None
    data['trade_type'] = trade_type
    final_capital = initial_capital
    trades = 0

    # Loop through the data to find trades
    for i in range(len(data)):
        prev = data['signals'][:i].sum()
        entry = 0.0
        exit = 0.0
        quantity = 0.0
        trade_type = 'exit'

        if data.loc[i, 'signals'] != 0 or final_capital != 0:
            j = i + 1
            if data.loc[i, 'signals'] == 1 and prev == 0:  # Long entry
                while j < len(data) and (data.loc[j, 'signals'] not in [-1, -2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'long'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            elif data.loc[i, 'signals'] == -1 and prev == 0:  # Short entry
                while j < len(data) and (data.loc[j, 'signals'] not in [1, 2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'short'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            if j < len(data):  # Make sure we found an exit
                data.loc[i, 'entry'] = entry
                data.loc[i, 'exit'] = exit
                data.loc[i, 'quantity'] = quantity
                data.loc[i, 'trade_type'] = trade_type

                pnl = quantity * (exit - entry) if data.loc[i, 'signals'] > 0 else quantity * (entry - exit)
                transaction_cost = quantity * (entry) * 0.0015
                net_pnl = pnl - transaction_cost
                final_capital += net_pnl

                data.loc[i, 'pnl'] = pnl
                data.loc[i, 'Transaction_Cost'] = transaction_cost
                data.loc[i, 'Net_Pnl'] = net_pnl

    # Cumulative data
    data['Cumulative_PnL'] = data['pnl'].cumsum()
    data['Cumulative_Transaction_Cost'] = data['Transaction_Cost'].cumsum()
    data['Cumulative_Net_Pnl'] = data['Net_Pnl'].cumsum()

    # Calculate Buy-and-Hold and Equity curves
    buy_and_hold_start = data['close'].iloc[0]
    data['Buy_and_Hold'] = (data['close'] / buy_and_hold_start) * initial_capital
    data['Equity'] = initial_capital + data['Cumulative_Net_Pnl']

    # Calculate Drawdown
    data['Peak_Equity'] = data['Equity'].cummax()
    data['drawdown'] = (data['Equity'] - data['Peak_Equity']) / data['Peak_Equity']
    max_drawdown = data['drawdown'].min()

    # Save the updated DataFrame to a CSV file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"/content/Trade_Logs_{timestamp}.csv"
    data.to_csv(output_file, index=True)

    return data, output_file

# Final function to process the trading data
def backtest(csv_file_path, initial_capital=1000, start_date=None, end_date=None):
    # Plot signals
    plot_signals(csv_file_path)

    # Calculate PnL and filter data
    data, output_file = calculate_pnl_data(csv_file_path, initial_capital)

    if start_date:
        data = data[data['datetime'] >= start_date]
    if end_date:
        data = data[data['datetime'] <= end_date]

    print(f"Output File: {output_file}")

In [146]:
def perform_backtest(csv_file_path):

    # Create an instance of the untrade client
    client = Client()

    # Perform backtest using the provided CSV file path
    result = client.backtest(
        jupyter_id="socks",  # the one you use to login to jupyter.untrade.io
        file_path=csv_file_path,
        leverage=1,  # Adjust leverage as needed
    )

    return result

if __name__ == "__main__":

    # Prompt user to upload a file in Google Colab
    # print("Please upload your CSV file:")
    # uploaded = files.upload()

    # Get the uploaded file path
    # csv_file_path = list(uploaded.keys())[0]
    # print(f"File uploaded successfully: {csv_file_path}")

    csv_file_path = '/content/result_df - 2024-11-11T173208.119.csv'

    # Perform backtest using the uploaded CSV file
    backtest_result = perform_backtest(csv_file_path)

    # Display the backtest result
    print("Backtest results:")
    for value in backtest_result:
        print(value)

    backtest(csv_file_path)


Backtest results:
data: {
  "jupyter_id": "socks",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2020-01-01 00:00:00",
      "Total Trades": 193,
      "Leverage Applied": 1.0,
      "Winning Trades": 143,
      "Losing Trades": 50,
      "No. of Long Trades": 127,
      "No. of Short Trades": 66,
      "Benchmark Return(%)": 1649.242832,
      "Benchmark Return(on $1000)": 16492.428319,
      "Win Rate": 74.093264,
      "Winning Streak": 14,
      "Losing Streak": 3,
      "Gross Profit": 8586.376504,
      "Net Profit": 8296.876504,
      "Average Profit": 42.988997,
      "Maximum Drawdown(%)": 9.965907,
      "Average Drawdown(%)": 0.812199,
      "Largest Win": 98.5,
      "Average Win": 84.619009,
      "Largest Loss": -329.302059,
      "Average Loss": -76.072836,
      "Maximum Holding Time": "37 days 19:59:59",
      "Average Holding Time": "3 days 10:25:29",
      "Maximum Adverse Excursion": 40.939287,
  

Output File: /content/Trade_Logs_20241111_120706.csv


# WE HAVE CLEANED THE SIGNALS BUT CAUSE OF STOP LOSS, IT LOOKS LIKE WE HAVENT, WE HAVENT IMPLEMENTED STOP LOSS IN OUR TRADE SIGNALS